In [1]:
%load_ext autoreload
%autoreload 2
# this only works on startup!
from jax import config
config.update("jax_enable_x64", True)

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = '1'



from encoding_information.gpu_utils import limit_gpu_memory_growth
limit_gpu_memory_growth()

from cleanplots import *
from tqdm import tqdm
from signal_utils_1D import *
from plot_utils_1D import *

2024-01-24 08:59:39.194874: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from scipy.signal import resample

N_objects = 5000
noise_sigma = 1e-3
num_nyquist_samples_list = [4, 9, 16, 25, 36]
num_bootstrap_samples = 11
upsampling_factor = 64
max_epochs = 5000
confidence = 90

object_names = ['white_noise', 'delta_uniform', '8_deltas_uniform']

num_nyquist_samples = NUM_NYQUIST_SAMPLES


def white_noise_fn(upsampling_factor):
  # generate a random object at the lowest resolution
  objects = []
  for i in range(N_objects):  
    object = generate_random_object('white_noise', object_size=upsampling_factor*num_nyquist_samples_list[0])
    # upsample it
    object = resample(object, upsampled_signal_length)
    objects.append(object)
  # print(objects[0].shape)
  return np.array(objects) 

mi_by_object = {}
mi_by_object_hi = {}
mi_by_object_lo = {}
for name in object_names:
  print(name)
  mutual_information = []
  mi_hi = []
  mi_lo = []
  for num_nyquist_samples in tqdm(num_nyquist_samples_list):
    upsampled_signal_length = upsampling_factor*num_nyquist_samples
    
    if name == 'delta_uniform':
      objects_fn = lambda : np.array([generate_random_object('delta', num_deltas=1, object_size=upsampled_signal_length) for i in range(N_objects)])
    elif name == '8_deltas_uniform':
      objects_fn = lambda :np.array([generate_random_object('delta', num_deltas=8, object_size=upsampled_signal_length) for i in range(N_objects)])
    else:
      objects_fn = lambda : white_noise_fn(upsampling_factor)

    runs = []
    for i in range(num_bootstrap_samples):
      initial_kernel, initial_params, optimized_params, objects, initial_mi, optimized_mi = optimize_PSF_and_estimate_mi(
          objects_fn, noise_sigma, num_nyquist_samples=num_nyquist_samples, upsampled_signal_length=upsampling_factor*num_nyquist_samples,
          max_epochs=max_epochs, 
          )
      runs.append(optimized_mi)
    runs = np.array(runs)
    mutual_information.append(np.median(runs))
    mi_lo.append(np.percentile(np.array(runs), 100-confidence))
    mi_hi.append(np.percentile(np.array(runs), confidence))
  mi_by_object[name] = mutual_information
  mi_by_object_hi[name] = mi_hi
  mi_by_object_lo[name] = mi_lo

white_noise


  0%|                                        | 0/5 [00:00<?, ?it/s]

initial loss -1.4351245316175762
computing optimized mi


2024-01-24 09:01:20.304945: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


initial loss -1.2294846109158586
computing optimized mi
initial loss -1.3213838476841826
computing optimized mi
initial loss -1.0855683726885292
computing optimized mi
initial loss -1.0276858143730252
computing optimized mi
initial loss -1.0972681087806002
computing optimized mi
initial loss -1.4877647263634874
computing optimized mi
initial loss -0.9928969192768811
computing optimized mi
initial loss -1.5548116721162595
computing optimized mi
initial loss -1.0282159284470265
computing optimized mi
initial loss -1.0313190870471731
computing optimized mi


...

/home/hpinkard_waller/GitRepos/EncodingInformation/encoding_information/information_estimation.py:226: UserWarning: 0.04% of pixels are negative.
  warnings.warn(f"{np.sum(clean_images_if_available < 0) / clean_images_if_available.size:.2%} of pixels are negative.")


initial loss -1.4506514212487338
computing optimized mi
initial loss -1.8453454017304212
computing optimized mi
initial loss -1.8870975574042705
computing optimized mi


/home/hpinkard_waller/GitRepos/EncodingInformation/encoding_information/information_estimation.py:226: UserWarning: 0.05% of pixels are negative.
  warnings.warn(f"{np.sum(clean_images_if_available < 0) / clean_images_if_available.size:.2%} of pixels are negative.")


initial loss -1.8482873517818998
computing optimized mi
initial loss -1.7669642897791678
computing optimized mi
initial loss -1.4238202746470014
computing optimized mi
initial loss -1.646104434855656
computing optimized mi
initial loss -1.6031025332521258
computing optimized mi
initial loss -1.3836126205363553
computing optimized mi


100%|████████████████████████████| 5/5 [7:50:54<00:00, 5650.95s/it]


In [3]:
fig, ax = plt.subplots(1, 1, figsize=(3.5, 3.5))

for object_name, mutual_information in mi_by_object.items():

    total_mi = np.array(mutual_information) * np.array(num_nyquist_samples_list)

    total_mi_hi = np.array(mi_by_object_hi[object_name]) * np.array(num_nyquist_samples_list)
    total_mi_lo = np.array(mi_by_object_lo[object_name]) * np.array(num_nyquist_samples_list)

    ax.plot(num_nyquist_samples_list, total_mi, 'o-', label=object_name)
    ax.fill_between(num_nyquist_samples_list, total_mi_lo, total_mi_hi, alpha=0.2)
    ax.set_xlabel('Signal bandwidth (number of Nyquist samples)')
    ax.set_ylabel('Mutual information (bits)')

    ax.set(ylim=(0, max(1.1 * np.max(total_mi), ax.get_ylim()[1])))
    # ax.set(ylim=[0, 25])

clear_spines(ax)
ax.legend()

fig.savefig('/home/hpinkard_waller/figures/1d_signals/' + 'mi_vs_bandwidth' + '.pdf', transparent=True)


In [4]:
# fig, ax = plt.subplots(1, 1, figsize=(3.5, 3.5))

# for object_name, mutual_information in mi_by_object.items():

#     total_mi = np.array(mutual_information) * np.array(num_nyquist_samples_list)
#     ax.semilogx(num_nyquist_samples_list, total_mi, 'o-', label=object_name)
#     ax.set_xlabel('Signal bandwidth (number of Nyquist samples)')
#     ax.set_ylabel('Mutual information (bits)')

#     ax.set(ylim=(0, max(1.1 * np.max(total_mi), ax.get_ylim()[1])))

# clear_spines(ax)
# ax.legend()

# fig.savefig('/home/hpinkard_waller/figures/1d_signals/' + 'mi_vs_bandwidth' + '.pdf', transparent=True)
